In [ ]:
import subprocess
import json
from urllib.request import urlopen

def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def install_dependencies_from_pyproject():
    # URL to raw pyproject.toml file in your GitHub repository
    pyproject_url = "https://raw.githubusercontent.com/rdhyee/isamples-python/exploratory/pyproject.toml"
    
    with urlopen(pyproject_url) as response:
        pyproject_content = response.read().decode()
    
    # Parse the TOML content
    import toml
    pyproject_data = toml.loads(pyproject_content)
    
    # Extract dependencies
    dependencies = pyproject_data.get('tool', {}).get('poetry', {}).get('dependencies', {})
    
    # Install each dependency
    for package, version in dependencies.items():
        if isinstance(version, str):
            subprocess.run(['pip', 'install', f"{package}{version}"])
        elif isinstance(version, dict):
            # Handle more complex version specifications
            version_str = version.get('version', '')
            subprocess.run(['pip', 'install', f"{package}{version_str}"])

if in_colab():
    # Install toml parser first
    subprocess.run(['pip', 'install', 'toml'])
    install_dependencies_from_pyproject()
    # pip install git+https://github.com/rdhyee/isamples-python.git@exploratory#egg=isamples_client
    subprocess.run(['pip', 'install', 'git+https://github.com/rdhyee/isamples-python.git@exploratory#egg=isamples_client'])

    from google.colab import output
    output.enable_custom_widget_manager()

In [ ]:
import duckdb

# Connect to a database (in-memory for this example)
con = duckdb.connect(database=':memory:', read_only=False)

# Execute the SQL commands
con.execute("SET VARIABLE parquet_path = 'https://z.rslv.xyz/10.5281/zenodo.15278210/isamples_export_2025_04_21_16_23_46_geo.parquet';")
con.execute("CREATE TEMP VIEW my_data AS SELECT(*) FROM read_parquet(getvariable('parquet_path'));")
result = con.execute("SELECT count(*) from my_data;").fetchone()

# Print the result
print(result[0])

# Close the connection
con.close()